# Custom templates


Prompt templates that automatically bind to a database are built on top of a specialized kind of prompt template in LangChain called `DependencyfulPromptTemplate`. This kind of prompt has two abilities:

1. it retain "dependencies" (such as a database connection) throughout its lifetime;
2. it admits named arguments to the `format` method that will be routed to its own "getter" function, instead of directly replacing the placeholder variables in the prompt string.

The Cassandra-backed prompt template then builds a certain `DependencyfulPromptTemplate` bound to the Cassandra database in the right way. Thanks to metadata inspection, detailed knowledge about the DB schema is not provided explicitly when creating the prompt, as was shown earlier.

As another example, the Feast template (shown in a later page) builds a different `DependencyfulPromptTemplate` which "knows" how to connect to Feast and retrieve the needed information to later fill the template string.

### Customization

However, sometimes one may need to go beyond the ready-made templates and build a custom prompt. This page shows how this is done.

Let's build a mock "account balance checker" that associates a current balance to each user (a key-value store), but it also keeps a counter of how many times a certain balance is accessed.

In [1]:
class Balancer():
    def __init__(self, accounts):
        self.accounts = accounts
        self._accesses = {}
        
    def read(self, user):
        if user in self.accounts:
            self._accesses[user] = self._accesses.get(user, 0) + 1
        return self.accounts[user]
    
    def accesses(self):
        return self._accesses

In [ ]:
bal0 = Balancer({'Rita': 100, 'Otto': 40, 'Anna': 150})
bal0.read('Rita')
bal0.read('Rita')
bal0.read('Anna')
print(bal0.accesses())

Ok, now let's create another balancer for use in the template:

In [2]:
myBalance = Balancer({'Katie': 1000, 'Gil': 800, 'Hugh': 1200})

### Template creation

#### The template string

In [3]:
from langchain.prompts.database.convertor_prompt_template import ConvertorPromptTemplate

In [4]:
balancePrompt = ("Hello. Your balance is {balance}. "
                 "You have {unreads} unread message(s).")

#### The getter and the convertor-prompt-template

In [7]:
def bal_getter(args_dict):
    user_name = args_dict['user_name']
    return {
        'balance': myBalance.read(user_name),
    }

In [11]:
myCustomTemplate = ConvertorPromptTemplate(
    template=balancePrompt,
    input_variables=['unreads'],
    convertor=bal_getter,
    convertor_input_variables=['user_name'],
    convertor_output_variables=['balance'],
)

### Rendering

In [12]:
myCustomTemplate.format(user_name='Gil', unreads=123)

'Hello. Your balance is 800. You have 123 unread message(s).'

In [13]:
myCustomTemplate.format(user_name='Gil', unreads=456)

'Hello. Your balance is 800. You have 456 unread message(s).'

In [14]:
myCustomTemplate.format(user_name='Katie', unreads=789)

'Hello. Your balance is 1000. You have 789 unread message(s).'

In [15]:
myBalance.accesses()

{'Gil': 2, 'Katie': 1}